In [1]:
# Cell 0: Install Required Libraries
# ----------------------------------
import sys
import subprocess

def install_package(package):
    try:
        __import__(package)
        print(f"✅ {package} already installed")
    except ImportError:
        print(f"📦 Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} installed successfully")

# Install required packages
install_package('pandas')
install_package('numpy') 
install_package('openpyxl')  # For Excel files

print("\n🚀 All packages ready!")

✅ pandas already installed
✅ numpy already installed
✅ openpyxl already installed

🚀 All packages ready!


In [2]:
# Cell 1: Import libraries
import pandas as pd
import numpy as np

print("📚 Libraries imported successfully!")

📚 Libraries imported successfully!


In [3]:
# Cell 2: Read your CSV file
# UPDATE THIS PATH TO YOUR ACTUAL FILE:
# file_path = "/Users/darrenang/Downloads/Backtest1.csv"  # ← Change this to your CSV path
file_path = r"C:\Users\yyang\OneDrive - SRM INTEGRATION (MALAYSIA) Sdn Bhd\Documents\TestT2\Backtester\Backtest1.csv"  # ← Change this to your CSV path

# Read the CSV
df = pd.read_csv(file_path)

print(f"✅ File loaded successfully!")
print(f"📊 Shape: {df.shape[0]} rows × {df.shape[1]} columns")

✅ File loaded successfully!
📊 Shape: 43 rows × 59 columns


In [4]:
# Cell 3: Show all column names
print("📋 ALL COLUMNS IN YOUR CSV:")
print("=" * 50)

for i, column in enumerate(df.columns, 1):
    print(f"{i:2d}. {column}")

print(f"\n📊 Total: {len(df.columns)} columns")

📋 ALL COLUMNS IN YOUR CSV:
 1. 1 R X Red Line
 2. Atr Track Back (After Atr)
 3. Atr Track Back (Multiplier)
 4. Account Name
 5. Adjusted Cost
 6. Adjusted Proceeds
 7. Avg Buy Price
 8. Avg Sell Price
 9. Exit Efficiency
10. Best Exit
11. Best Exit Price
12. Best Exit Time
13. Close Date
14. Close Time
15. Commission
16. Duration
17. Entry Candle Size And Color
18. Entry Price
19. Executions
20. Exit Price
21. Gold Candle
22. Gross P&L
23. Trade Risk
24. Initial Target
25. Instrument
26. Spread Type
27. M5, M30, 1 H
28. Macd Condition
29. Max Tp (Atr Multiplier)
30. Mistakes
31. Net P&L
32. Net ROI
33. Open Date
34. Open Time
35. Pips
36. Reward Ratio
37. Playbook
38. Points
39. Position MAE
40. Position MFE
41. Price MAE
42. Price MFE
43. Rsi Position
44. Realized RR
45. Return Per Pip
46. Reviewed
47. Sl Track Back Behind Redline
48. Setups
49. Side
50. Status
51. Symbol
52. Ticks Value
53. Ticks Per Contract
54. Fee
55. Swap
56. Rating
57. Type Of Trades
58. Quantity
59. Zella Sco

In [5]:
# Extract the three key columns
key_columns = [
    'Atr Track Back (Multiplier)',
    'Max Tp (Atr Multiplier)', 
    '1 R X Red Line'  # NEW: Realistic exit data
]

# Create enhanced dataframe
enhanced_df = df[key_columns].copy()

print("🎯 EXTRACTED ENHANCED DATA:")
print("=" * 50)
print(f"📊 Shape: {enhanced_df.shape[0]} rows × {enhanced_df.shape[1]} columns")
print(f"📋 Columns: {list(enhanced_df.columns)}")

🎯 EXTRACTED ENHANCED DATA:
📊 Shape: 43 rows × 3 columns
📋 Columns: ['Atr Track Back (Multiplier)', 'Max Tp (Atr Multiplier)', '1 R X Red Line']


In [6]:
# Analyze what values you have in Red Line column
print("📊 RED LINE VALUES ANALYSIS:")
print("=" * 50)

red_line_values = enhanced_df['1 R X Red Line'].value_counts()
print("Raw values and counts:")
print(red_line_values)

print(f"\n📈 UNIQUE RED LINE VALUES:")
unique_reds = enhanced_df['1 R X Red Line'].unique()
for value in unique_reds:
    if pd.notna(value):
        print(f"   • '{value}'")

📊 RED LINE VALUES ANALYSIS:
Raw values and counts:
1 R X Red Line
lose lose                         17
win win                           13
win lose                           6
win second win if no breakeven     4
Name: count, dtype: int64

📈 UNIQUE RED LINE VALUES:
   • 'win win'
   • 'win lose'
   • 'lose lose'
   • 'win second win if no breakeven'


In [7]:
# Clean all three columns
def clean_atr_value(value):
    """Convert ATR values to numbers"""
    if pd.isna(value):
        return None
    
    value_str = str(value).strip()
    
    if value_str == "-1" or "lost trade" in value_str.lower():
        return -1.0
    
    if value_str.endswith('x'):
        try:
            return float(value_str[:-1])
        except:
            return None
    
    try:
        return float(value_str)
    except:
        return None

def clean_red_line_value(value):
    """Clean Red Line values"""
    if pd.isna(value):
        return None
    
    value_str = str(value).strip().lower()
    
    # Map your categories
    if "win win" in value_str:
        return "win_win"
    elif "win lose" in value_str:
        return "win_lose"  
    elif "win second win if no breakeven" in value_str:
        return "win_win"  # You said treat this as win_win
    elif "lose" in value_str or "lost" in value_str:
        return "lose"
    else:
        return value_str  # Keep original if unrecognized

print("🧹 CLEANING ALL DATA...")

# Clean the data
enhanced_df['Trackback_Clean'] = enhanced_df['Atr Track Back (Multiplier)'].apply(clean_atr_value)
enhanced_df['MaxTP_Clean'] = enhanced_df['Max Tp (Atr Multiplier)'].apply(clean_atr_value)
enhanced_df['RedLine_Clean'] = enhanced_df['1 R X Red Line'].apply(clean_red_line_value)

# Filter out invalid rows
valid_enhanced = enhanced_df[
    enhanced_df['MaxTP_Clean'].notna() & 
    enhanced_df['RedLine_Clean'].notna()
].copy()

print("✅ Data cleaned!")
print(f"📊 Valid trades: {len(valid_enhanced)}")

# Show the breakdown
print(f"\n📊 RED LINE BREAKDOWN:")
red_breakdown = valid_enhanced['RedLine_Clean'].value_counts()
print(red_breakdown)

print(f"\n👀 SAMPLE OF CLEANED DATA:")
print(valid_enhanced[['Atr Track Back (Multiplier)', 'Trackback_Clean', 
                     'Max Tp (Atr Multiplier)', 'MaxTP_Clean',
                     '1 R X Red Line', 'RedLine_Clean']].head(10))

🧹 CLEANING ALL DATA...
✅ Data cleaned!
📊 Valid trades: 40

📊 RED LINE BREAKDOWN:
RedLine_Clean
win_win     17
lose        17
win_lose     6
Name: count, dtype: int64

👀 SAMPLE OF CLEANED DATA:
   Atr Track Back (Multiplier)  Trackback_Clean Max Tp (Atr Multiplier)  \
0                         0.5x              0.5                    1.0x   
1                         0.5x              0.5                    2.1x   
2                         0.5x              0.5                    2.8x   
3                -1 lost trade             -1.0                      -1   
4                -1 lost trade             -1.0                      -1   
6                -1 lost trade             -1.0                      -1   
7                -1 lost trade             -1.0                      -1   
8                -1 lost trade             -1.0                      -1   
9                         0.5x              0.5                    1.6x   
10                        0.5x              0.5          

### Latest Simulation

In [17]:
# Complete ATR + R Target Optimization with FULL Logging for ALL Combinations
import pandas as pd
import numpy as np
import sys
from datetime import datetime

def simulate_atr_and_r_target_with_logging(data, atr_stop_level, r_target, log_file=None):
    """
    Simulate performance with detailed logging for verification
    r_target can be:
    - A number (e.g., 2.0) for fixed R target strategy
    - "redline_clean_X.X" (e.g., "redline_clean_2.4") for redline strategy with X.X R target (in current ATR terms)
    """
    results = []
    trade_logs = []
    
    # Parse target type
    if isinstance(r_target, str) and r_target.startswith("redline_clean_"):
        is_redline = True
        target_r_current_atr = float(r_target.split("_")[2])  # This is already in current ATR terms
        # Convert to original 1.0x ATR terms for comparison with MaxTP
        original_win_lose_r = target_r_current_atr * atr_stop_level
        target_display = f"Redline({target_r_current_atr:.1f}R)"
    else:
        is_redline = False
        original_win_lose_r = None
        target_r_current_atr = r_target  # Already in current ATR terms
        target_display = f"{r_target:.1f}R"
    
    if log_file:
        log_file.write(f"\n🔍 COMBINATION: {atr_stop_level:.1f}x ATR + {target_display} target\n")
        log_file.write("=" * 100 + "\n")
        log_file.write(f"{'Trade':<5} {'Orig_TB':<8} {'Orig_MaxTP':<10} {'Orig_RedLine':<12} {'Logic Applied':<45} {'Result':<8}\n")
        log_file.write("-" * 100 + "\n")
    
    for idx, trade in data.iterrows():
        trackback = trade['Trackback_Clean']
        max_tp = trade['MaxTP_Clean']
        red_line = trade['RedLine_Clean']
        
        if pd.isna(trackback) or pd.isna(max_tp) or pd.isna(red_line):
            continue
        
        # Initialize variables for logging
        logic_applied = ""
        
        # Simulation logic with detailed logging
        if red_line == "lose":
            # Original loser - check if wider stop saves it
            if trackback != -1.0 and trackback < atr_stop_level:
                # Could be saved, but what would the result be?
                if is_redline:
                    # For redline, if saved, what would the result be?
                    # We treat saved losers as potential win_lose trades
                    if max_tp >= original_win_lose_r:  # Compare in original ATR terms!
                        risk_ratio = target_r_current_atr  # Take the target in current ATR terms
                        # Show comparison in current ATR terms for clarity
                        max_tp_current_atr = max_tp / atr_stop_level
                        logic_applied = f"Saved, MaxTP {max_tp_current_atr:.1f}x >= {target_r_current_atr:.1f}x target, take {target_r_current_atr:.1f}R"
                    else:
                        risk_ratio = -1.0  # Saved but didn't reach win_lose target in ATR terms
                        # Show comparison in current ATR terms for clarity
                        max_tp_current_atr = max_tp / atr_stop_level
                        logic_applied = f"Saved but MaxTP {max_tp_current_atr:.1f}x < {target_r_current_atr:.1f}x target"
                else:
                    # Fixed R target
                    potential_max = max_tp / atr_stop_level if max_tp > 0 else 0
                    if potential_max >= r_target:
                        risk_ratio = r_target  # Hit our R target
                        logic_applied = f"Saved by wide stop, hit {r_target:.1f}R target"
                    else:
                        risk_ratio = -1.0  # Saved but didn't reach target
                        logic_applied = f"Saved but missed target: {potential_max:.2f} < {r_target:.1f}R"
            else:
                risk_ratio = -1.0  # Still a loser
                if trackback == -1.0:
                    logic_applied = "Original loser (-1), still loses"
                else:
                    logic_applied = f"Trackback {trackback:.1f} >= stop {atr_stop_level:.1f}, stopped"
                
        else:
            # Original winner (win_win or win_lose)
            if trackback >= atr_stop_level:
                risk_ratio = -1.0  # New stop would stop us out
                logic_applied = f"Stopped out: trackback {trackback:.1f} >= stop {atr_stop_level:.1f}"
            else:
                # Trade survives, calculate result based on target type
                if is_redline:
                    # Use enhanced redline strategy
                    if red_line == "win_win":
                        # win_win means they actually reached their ORIGINAL redline and exited
                        # But with new strategy, would they exit earlier at the new target?
                        actual_exit_scaled = max_tp / atr_stop_level
                        
                        if actual_exit_scaled >= target_r_current_atr:
                            # They would exit earlier at new target, not wait for actual exit
                            risk_ratio = target_r_current_atr
                            logic_applied = f"Redline win_win EARLY EXIT: Would exit at {target_r_current_atr:.1f}R target (vs {actual_exit_scaled:.1f}R actual)"
                        else:
                            # New target higher than actual exit - they still exit at actual
                            risk_ratio = actual_exit_scaled
                            logic_applied = f"Redline win_win ACTUAL EXIT: Target {target_r_current_atr:.1f}R > actual {actual_exit_scaled:.1f}R"
                    elif red_line == "win_lose":
                        # win_lose means they missed original redline, only got 1R
                        # Now check if they would reach our NEW target
                        if max_tp >= original_win_lose_r:  # Compare in original ATR terms!
                            risk_ratio = target_r_current_atr  # Take the target in current ATR terms
                            # Show comparison in current ATR terms for clarity
                            max_tp_current_atr = max_tp / atr_stop_level
                            logic_applied = f"Redline win_lose HIT: MaxTP {max_tp_current_atr:.1f}x >= {target_r_current_atr:.1f}x target, take {target_r_current_atr:.1f}R"
                        else:
                            risk_ratio = -1.0  # Didn't reach our win_lose target in original ATR terms
                            # Show comparison in current ATR terms for clarity
                            max_tp_current_atr = max_tp / atr_stop_level
                            logic_applied = f"Redline win_lose MISS: MaxTP {max_tp_current_atr:.1f}x < {target_r_current_atr:.1f}x target"
                else:
                    # Fixed R target strategy
                    potential_max = max_tp / atr_stop_level
                    
                    if potential_max >= r_target:
                        risk_ratio = r_target  # Hit our R target
                        logic_applied = f"Hit R target: {potential_max:.2f} >= {r_target:.1f}, take {r_target:.1f}R"
                    else:
                        # Didn't reach our R target - NO PROFIT TAKEN
                        risk_ratio = -1.0  # Either stopped out or didn't reach target
                        logic_applied = f"Missed R target: {potential_max:.2f} < {r_target:.1f}R, no profit"
        
        results.append(risk_ratio)
        
        # Store detailed log
        trade_log = {
            'Trade_Idx': idx,
            'ATR_Stop': atr_stop_level,
            'R_Target': target_display,
            'Original_Trackback': trackback,
            'Original_MaxTP': max_tp,
            'Original_RedLine': red_line,
            'Logic_Applied': logic_applied,
            'Final_RR': risk_ratio
        }
        trade_logs.append(trade_log)
        
        if log_file:
            log_file.write(f"{idx:<5} {trackback:<8} {max_tp:<10} {red_line:<12} {logic_applied:<45} {risk_ratio:+8.2f}\n")
    
    # Calculate summary metrics
    total_trades = len(results)
    winners = [rr for rr in results if rr > 0]
    losers = [rr for rr in results if rr <= 0]
    
    win_rate = len(winners) / total_trades * 100 if total_trades > 0 else 0
    avg_winner = np.mean(winners) if winners else 0
    avg_loser = abs(np.mean(losers)) if losers else 1
    expectancy = np.mean(results) if results else 0
    total_profit = sum(winners) if winners else 0
    profit_factor = total_profit / abs(sum(losers)) if losers else 0
    
    if log_file:
        log_file.write(f"\n📊 SUMMARY:\n")
        log_file.write(f"   • Total Trades: {total_trades}\n")
        log_file.write(f"   • Winners: {len(winners)} | Losers: {len(losers)}\n")
        log_file.write(f"   • Win Rate: {win_rate:.1f}%\n")
        log_file.write(f"   • Avg Winner: {avg_winner:.3f}R\n")
        log_file.write(f"   • Avg Loser: {avg_loser:.3f}R\n")
        log_file.write(f"   • Total Profit: {total_profit:.2f}R\n")
        log_file.write(f"   • Expectancy: {expectancy:+.3f}R per trade\n")
        log_file.write(f"   • Profit Factor: {profit_factor:.2f}\n")
        log_file.write("\n" + "="*100 + "\n")
    
    return {
        'ATR_Stop': atr_stop_level,
        'R_Target': target_display,
        'Win_Rate': win_rate,
        'Avg_Winner': avg_winner,
        'Avg_Loser': avg_loser,
        'Expectancy': expectancy,
        'Profit_Factor': profit_factor,
        'Total_Profit': total_profit,
        'Winners': len(winners),
        'Losers': len(losers),
        'Trade_Logs': trade_logs
    }

# Complete optimization ranges - Simplified approach for clean increments
atr_levels = np.arange(0.6, 3.1, 0.1).round(1).tolist()  # 0.6 to 3.0 in 0.1 increments

# Define clean target ranges for both redline and fixed strategies
clean_targets = np.arange(1.0, 5.1, 0.2).round(1).tolist()  # 1.0, 1.2, 1.4, ... 5.0

# Generate all combinations
all_combinations = []

for atr_level in atr_levels:
    # Add redline strategies with clean targets (in current ATR terms)
    for target_r in clean_targets:
        redline_strategy = f"redline_clean_{target_r:.1f}"
        all_combinations.append((atr_level, redline_strategy))
    
    # Add fixed R targets
    for fixed_r in clean_targets:
        all_combinations.append((atr_level, fixed_r))

total_combinations = len(all_combinations)

print(f"🚀 RUNNING COMPLETE OPTIMIZATION WITH FULL LOGGING")
print(f"📊 Testing {total_combinations} combinations")
print(f"🎯 Clean target increments: 1.0R, 1.2R, 1.4R, etc. (no decimals like 4.67R)")
print(f"   • Redline strategies: win_win=actual exits, win_lose=test new targets") 
print(f"   • Fixed R targets: Binary outcome (hit target or -1R)")
print(f"📁 This will generate detailed logs for ALL combinations...")

# Create timestamped filenames
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
detailed_log_filename = f"complete_atr_optimization_detailed_{timestamp}.txt"
summary_filename = f"complete_atr_optimization_summary_{timestamp}.txt"
results_csv_filename = f"complete_atr_optimization_results_{timestamp}.csv"

print(f"📝 Detailed log: {detailed_log_filename}")
print(f"📊 Summary: {summary_filename}")
print(f"📈 Results CSV: {results_csv_filename}")

# Run complete optimization with logging
optimization_results = []

# Open detailed log file
with open(detailed_log_filename, 'w', encoding='utf-8') as detailed_log:
    detailed_log.write("🔍 COMPLETE ATR + R TARGET OPTIMIZATION - DETAILED LOG\n")
    detailed_log.write("=" * 80 + "\n")
    detailed_log.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    detailed_log.write(f"Dataset: {len(valid_enhanced)} valid trades\n")
    detailed_log.write(f"ATR Range: {atr_levels[0]:.1f} to {atr_levels[-1]:.1f} (increments: 0.1)\n")
    detailed_log.write(f"Target Range: {clean_targets[0]:.1f} to {clean_targets[-1]:.1f} (increments: 0.2)\n")
    detailed_log.write(f"Total Combinations: {total_combinations}\n")
    detailed_log.write("=" * 80 + "\n")
    detailed_log.write("🎯 CORRECT REDLINE LOGIC:\n")
    detailed_log.write("   • win_win trades = actual exits at original redline (get full profit)\n") 
    detailed_log.write("   • win_lose trades = missed original redline (test against new target)\n")
    detailed_log.write("   • Targets shown in current ATR terms (clean increments)\n")
    detailed_log.write("🎯 FIXED R TARGETS: Binary outcome (hit exact target = +R, miss target = -1R)\n")
    detailed_log.write("=" * 80 + "\n\n")
    
    # Run all combinations
    for combination_count, (atr_level, r_target) in enumerate(all_combinations, 1):
        
        # Show progress
        if combination_count % 100 == 0:
            print(f"📈 Progress: {combination_count}/{total_combinations} combinations completed ({combination_count/total_combinations*100:.1f}%)")
        
        # Run simulation with logging
        result = simulate_atr_and_r_target_with_logging(
            valid_enhanced, atr_level, r_target, detailed_log
        )
        
        optimization_results.append(result)

print(f"✅ Completed all {total_combinations} combinations!")

# Convert to DataFrame for analysis
opt_df = pd.DataFrame([{k: v for k, v in result.items() if k != 'Trade_Logs'} 
                       for result in optimization_results])

# Save results to CSV
opt_df.to_csv(results_csv_filename, index=False)

# DEFINE ALL VARIABLES BEFORE USING THEM
# Sort all results by expectancy (highest to lowest)
all_by_expectancy = opt_df.sort_values('Expectancy', ascending=False).reset_index(drop=True)

# Find best results for each metric
best_expectancy = opt_df.loc[opt_df['Expectancy'].idxmax()]
best_profit = opt_df.loc[opt_df['Total_Profit'].idxmax()]
best_winrate = opt_df.loc[opt_df['Win_Rate'].idxmax()]

# Top results for reference sections
top_20_expectancy = opt_df.nlargest(20, 'Expectancy')
top_20_profit = opt_df.nlargest(20, 'Total_Profit')
top_20_winrate = opt_df.nlargest(20, 'Win_Rate')

# Create summary file
with open(summary_filename, 'w', encoding='utf-8') as summary:
    summary.write("🎯 COMPLETE ATR + R TARGET OPTIMIZATION - SUMMARY\n")
    summary.write("=" * 60 + "\n")
    summary.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    summary.write(f"Total combinations tested: {total_combinations}\n")
    summary.write(f"Dataset: {len(valid_enhanced)} valid trades\n")
    summary.write(f"Clean target increments: 1.0R, 1.2R, 1.4R, etc.\n\n")
    
    summary.write("🎯 TARGET EXPLANATIONS:\n")
    summary.write("   • Redline(X.XR): win_win=actual exits, win_lose=test new target\n")
    summary.write("   • Fixed R: Binary outcome (hit exact target or -1R loss)\n\n")
    
    summary.write("🏆 OPTIMAL RESULTS:\n")
    summary.write("-" * 40 + "\n")
    summary.write(f"🎯 Best Expectancy: {best_expectancy['ATR_Stop']:.1f}x ATR + {best_expectancy['R_Target']} target\n")
    summary.write(f"   Expectancy: {best_expectancy['Expectancy']:+.3f}R per trade\n")
    summary.write(f"   Win Rate: {best_expectancy['Win_Rate']:.1f}%\n")
    summary.write(f"   Total Profit: {best_expectancy['Total_Profit']:.2f}R\n\n")
    
    summary.write(f"💰 Highest Total Profit: {best_profit['ATR_Stop']:.1f}x ATR + {best_profit['R_Target']} target\n")
    summary.write(f"   Total Profit: {best_profit['Total_Profit']:.2f}R\n")
    summary.write(f"   Expectancy: {best_profit['Expectancy']:+.3f}R\n")
    summary.write(f"   Win Rate: {best_profit['Win_Rate']:.1f}%\n\n")
    
    summary.write(f"🎪 Highest Win Rate: {best_winrate['ATR_Stop']:.1f}x ATR + {best_winrate['R_Target']} target\n")
    summary.write(f"   Win Rate: {best_winrate['Win_Rate']:.1f}%\n")
    summary.write(f"   Expectancy: {best_winrate['Expectancy']:+.3f}R\n")
    summary.write(f"   Total Profit: {best_winrate['Total_Profit']:.2f}R\n\n")
    
    # All combinations by expectancy
    summary.write("📊 ALL COMBINATIONS BY EXPECTANCY (COMPLETE LIST):\n")
    summary.write("-" * 90 + "\n")
    summary.write(f"{'Rank':<4} {'ATR':<6} {'Target':<15} {'Expectancy':<10} {'Win Rate':<8} {'Total Profit':<12} {'Winners':<7} {'Losers':<6}\n")
    summary.write("-" * 90 + "\n")
    
    for i, (_, row) in enumerate(all_by_expectancy.iterrows(), 1):
        target_str = str(row['R_Target'])[:14]  # Truncate if too long
        summary.write(f"{i:<4} {row['ATR_Stop']:<6.1f} {target_str:<15} {row['Expectancy']:<10.3f} {row['Win_Rate']:<8.1f}% {row['Total_Profit']:<12.2f} {row['Winners']:<7} {row['Losers']:<6}\n")

    # Add summary statistics
    summary.write(f"\n📈 SUMMARY STATISTICS:\n")
    summary.write("-" * 40 + "\n")
    summary.write(f"Best Expectancy: {opt_df['Expectancy'].max():.3f}\n")
    summary.write(f"Worst Expectancy: {opt_df['Expectancy'].min():.3f}\n")
    summary.write(f"Average Expectancy: {opt_df['Expectancy'].mean():.3f}\n")
    summary.write(f"Median Expectancy: {opt_df['Expectancy'].median():.3f}\n")
    summary.write(f"Std Dev Expectancy: {opt_df['Expectancy'].std():.3f}\n")
    
    summary.write(f"\nProfitable Combinations: {len(opt_df[opt_df['Expectancy'] > 0])}/{len(opt_df)} ({len(opt_df[opt_df['Expectancy'] > 0])/len(opt_df)*100:.1f}%)\n")
    summary.write(f"Expectancy > 0.20: {len(opt_df[opt_df['Expectancy'] > 0.20])}/{len(opt_df)} ({len(opt_df[opt_df['Expectancy'] > 0.20])/len(opt_df)*100:.1f}%)\n")
    summary.write(f"Expectancy > 0.15: {len(opt_df[opt_df['Expectancy'] > 0.15])}/{len(opt_df)} ({len(opt_df[opt_df['Expectancy'] > 0.15])/len(opt_df)*100:.1f}%)\n")
    
    # Top 10 by other metrics for reference
    summary.write(f"\n💰 TOP 10 BY TOTAL PROFIT:\n")
    summary.write("-" * 70 + "\n")
    for i, (_, row) in enumerate(top_20_profit.head(10).iterrows(), 1):
        target_str = str(row['R_Target'])[:14]
        summary.write(f"{i:<2} {row['ATR_Stop']:.1f}x {target_str:<15} | Profit: {row['Total_Profit']:.2f}R | Exp: {row['Expectancy']:+.3f}\n")
    
    summary.write(f"\n🎯 TOP 10 BY WIN RATE:\n")
    summary.write("-" * 70 + "\n")
    for i, (_, row) in enumerate(top_20_winrate.head(10).iterrows(), 1):
        target_str = str(row['R_Target'])[:14]
        summary.write(f"{i:<2} {row['ATR_Stop']:.1f}x {target_str:<15} | WR: {row['Win_Rate']:.1f}% | Exp: {row['Expectancy']:+.3f}\n")

print(f"\n🎊 COMPLETE OPTIMIZATION FINISHED!")
print("=" * 50)
print(f"📁 Files created:")
print(f"   • {detailed_log_filename} - Complete trade-by-trade logs")
print(f"   • {summary_filename} - ALL {total_combinations} combinations ranked by expectancy")
print(f"   • {results_csv_filename} - All results in spreadsheet format")

print(f"\n🏆 BEST RESULTS:")
print(f"🎯 Optimal: {best_expectancy['ATR_Stop']:.1f}x ATR + {best_expectancy['R_Target']} target (Expectancy: {best_expectancy['Expectancy']:+.3f})")
print(f"💰 Max Profit: {best_profit['ATR_Stop']:.1f}x ATR + {best_profit['R_Target']} target (Profit: {best_profit['Total_Profit']:.2f}R)")

print(f"\n📊 PERFORMANCE DISTRIBUTION:")
profitable_count = len(opt_df[opt_df['Expectancy'] > 0])
excellent_count = len(opt_df[opt_df['Expectancy'] > 0.20])
print(f"   • Profitable combinations: {profitable_count}/{total_combinations} ({profitable_count/total_combinations*100:.1f}%)")
print(f"   • Excellent (>0.20 expectancy): {excellent_count}/{total_combinations} ({excellent_count/total_combinations*100:.1f}%)")

print(f"\n✨ Check the summary file for complete ranking of all {total_combinations} combinations!")

🚀 RUNNING COMPLETE OPTIMIZATION WITH FULL LOGGING
📊 Testing 1050 combinations
🎯 Clean target increments: 1.0R, 1.2R, 1.4R, etc. (no decimals like 4.67R)
   • Redline strategies: win_win=actual exits, win_lose=test new targets
   • Fixed R targets: Binary outcome (hit target or -1R)
📁 This will generate detailed logs for ALL combinations...
📝 Detailed log: complete_atr_optimization_detailed_20250608_111716.txt
📊 Summary: complete_atr_optimization_summary_20250608_111716.txt
📈 Results CSV: complete_atr_optimization_results_20250608_111716.csv
📈 Progress: 100/1050 combinations completed (9.5%)
📈 Progress: 200/1050 combinations completed (19.0%)
📈 Progress: 300/1050 combinations completed (28.6%)
📈 Progress: 400/1050 combinations completed (38.1%)
📈 Progress: 500/1050 combinations completed (47.6%)
📈 Progress: 600/1050 combinations completed (57.1%)
📈 Progress: 700/1050 combinations completed (66.7%)
📈 Progress: 800/1050 combinations completed (76.2%)
📈 Progress: 900/1050 combinations comp

In [18]:
# Complete ATR + R Target Optimization with FULL Logging for ALL Combinations
import pandas as pd
import numpy as np
import sys
from datetime import datetime

def simulate_atr_and_r_target_with_logging(data, atr_stop_level, r_target, log_file=None):
    """
    Simulate performance with detailed logging for verification
    r_target can be:
    - A number (e.g., 2.0) for fixed R target strategy
    - "redline_clean_X.X" (e.g., "redline_clean_2.4") for redline strategy with X.X R target (in current ATR terms)
    """
    results = []
    trade_logs = []
    
    # Parse target type
    if isinstance(r_target, str) and r_target.startswith("redline_clean_"):
        is_redline = True
        target_r_current_atr = float(r_target.split("_")[2])  # This is already in current ATR terms
        # Convert to original 1.0x ATR terms for comparison with MaxTP
        original_win_lose_r = target_r_current_atr * atr_stop_level
        target_display = f"Redline({target_r_current_atr:.1f}R)"
    else:
        is_redline = False
        original_win_lose_r = None
        target_r_current_atr = r_target  # Already in current ATR terms
        target_display = f"{r_target:.1f}R"
    
    if log_file:
        log_file.write(f"\n🔍 COMBINATION: {atr_stop_level:.1f}x ATR + {target_display} target\n")
        log_file.write("=" * 100 + "\n")
        log_file.write(f"{'Trade':<5} {'Orig_TB':<8} {'Orig_MaxTP':<10} {'Orig_RedLine':<12} {'Logic Applied':<45} {'Result':<8}\n")
        log_file.write("-" * 100 + "\n")
    
    for idx, trade in data.iterrows():
        trackback = trade['Trackback_Clean']
        max_tp = trade['MaxTP_Clean']
        red_line = trade['RedLine_Clean']
        
        if pd.isna(trackback) or pd.isna(max_tp) or pd.isna(red_line):
            continue
        
        # Initialize variables for logging
        logic_applied = ""
        
        # Simulation logic with detailed logging
        if red_line == "lose":
            # Original loser - check if wider stop saves it
            if trackback != -1.0 and trackback < atr_stop_level:
                # Could be saved, but what would the result be?
                if is_redline:
                    # For redline, if saved, what would the result be?
                    # We treat saved losers as potential win_lose trades
                    if max_tp >= original_win_lose_r:  # Compare in original ATR terms!
                        risk_ratio = target_r_current_atr  # Take the target in current ATR terms
                        # Show comparison in current ATR terms for clarity
                        max_tp_current_atr = max_tp / atr_stop_level
                        logic_applied = f"Saved, MaxTP {max_tp_current_atr:.1f}x >= {target_r_current_atr:.1f}x target, take {target_r_current_atr:.1f}R"
                    else:
                        risk_ratio = -1.0  # Saved but didn't reach win_lose target in ATR terms
                        # Show comparison in current ATR terms for clarity
                        max_tp_current_atr = max_tp / atr_stop_level
                        logic_applied = f"Saved but MaxTP {max_tp_current_atr:.1f}x < {target_r_current_atr:.1f}x target"
                else:
                    # Fixed R target
                    potential_max = max_tp / atr_stop_level if max_tp > 0 else 0
                    if potential_max >= r_target:
                        risk_ratio = r_target  # Hit our R target
                        logic_applied = f"Saved by wide stop, hit {r_target:.1f}R target"
                    else:
                        risk_ratio = -1.0  # Saved but didn't reach target
                        logic_applied = f"Saved but missed target: {potential_max:.2f} < {r_target:.1f}R"
            else:
                risk_ratio = -1.0  # Still a loser
                if trackback == -1.0:
                    logic_applied = "Original loser (-1), still loses"
                else:
                    logic_applied = f"Trackback {trackback:.1f} >= stop {atr_stop_level:.1f}, stopped"
                
        else:
            # Original winner (win_win or win_lose)
            if trackback >= atr_stop_level:
                risk_ratio = -1.0  # New stop would stop us out
                logic_applied = f"Stopped out: trackback {trackback:.1f} >= stop {atr_stop_level:.1f}"
            else:
                # Trade survives, calculate result based on target type
                if is_redline:
                    # Use enhanced redline strategy
                    if red_line == "win_win":
                        # win_win means they actually reached their ORIGINAL redline and exited
                        # But with new strategy, would they exit earlier at the new target?
                        actual_exit_scaled = max_tp / atr_stop_level
                        
                        if actual_exit_scaled >= target_r_current_atr:
                            # They would exit earlier at new target, not wait for actual exit
                            risk_ratio = target_r_current_atr
                            logic_applied = f"Redline win_win EARLY EXIT: Would exit at {target_r_current_atr:.1f}R target (vs {actual_exit_scaled:.1f}R actual)"
                        else:
                            # New target higher than actual exit - they still exit at actual
                            risk_ratio = actual_exit_scaled
                            logic_applied = f"Redline win_win ACTUAL EXIT: Target {target_r_current_atr:.1f}R > actual {actual_exit_scaled:.1f}R"
                    elif red_line == "win_lose":
                        # win_lose means they missed original redline, only got 1R
                        # Now check if they would reach our NEW target
                        if max_tp >= original_win_lose_r:  # Compare in original ATR terms!
                            risk_ratio = target_r_current_atr  # Take the target in current ATR terms
                            # Show comparison in current ATR terms for clarity
                            max_tp_current_atr = max_tp / atr_stop_level
                            logic_applied = f"Redline win_lose HIT: MaxTP {max_tp_current_atr:.1f}x >= {target_r_current_atr:.1f}x target, take {target_r_current_atr:.1f}R"
                        else:
                            risk_ratio = -1.0  # Didn't reach our win_lose target in original ATR terms
                            # Show comparison in current ATR terms for clarity
                            max_tp_current_atr = max_tp / atr_stop_level
                            logic_applied = f"Redline win_lose MISS: MaxTP {max_tp_current_atr:.1f}x < {target_r_current_atr:.1f}x target"
                else:
                    # Fixed R target strategy
                    potential_max = max_tp / atr_stop_level
                    
                    if potential_max >= r_target:
                        risk_ratio = r_target  # Hit our R target
                        logic_applied = f"Hit R target: {potential_max:.2f} >= {r_target:.1f}, take {r_target:.1f}R"
                    else:
                        # Didn't reach our R target - NO PROFIT TAKEN
                        risk_ratio = -1.0  # Either stopped out or didn't reach target
                        logic_applied = f"Missed R target: {potential_max:.2f} < {r_target:.1f}R, no profit"
        
        results.append(risk_ratio)
        
        # Store detailed log
        trade_log = {
            'Trade_Idx': idx,
            'ATR_Stop': atr_stop_level,
            'R_Target': target_display,
            'Original_Trackback': trackback,
            'Original_MaxTP': max_tp,
            'Original_RedLine': red_line,
            'Logic_Applied': logic_applied,
            'Final_RR': risk_ratio
        }
        trade_logs.append(trade_log)
        
        if log_file:
            log_file.write(f"{idx:<5} {trackback:<8} {max_tp:<10} {red_line:<12} {logic_applied:<45} {risk_ratio:+8.2f}\n")
    
    # Calculate summary metrics
    total_trades = len(results)
    winners = [rr for rr in results if rr > 0]
    losers = [rr for rr in results if rr <= 0]
    
    win_rate = len(winners) / total_trades * 100 if total_trades > 0 else 0
    avg_winner = np.mean(winners) if winners else 0
    avg_loser = abs(np.mean(losers)) if losers else 1
    expectancy = np.mean(results) if results else 0
    total_profit = sum(winners) if winners else 0
    profit_factor = total_profit / abs(sum(losers)) if losers else 0
    
    if log_file:
        log_file.write(f"\n📊 SUMMARY:\n")
        log_file.write(f"   • Total Trades: {total_trades}\n")
        log_file.write(f"   • Winners: {len(winners)} | Losers: {len(losers)}\n")
        log_file.write(f"   • Win Rate: {win_rate:.1f}%\n")
        log_file.write(f"   • Avg Winner: {avg_winner:.3f}R\n")
        log_file.write(f"   • Avg Loser: {avg_loser:.3f}R\n")
        log_file.write(f"   • Total Profit: {total_profit:.2f}R\n")
        log_file.write(f"   • Expectancy: {expectancy:+.3f}R per trade\n")
        log_file.write(f"   • Profit Factor: {profit_factor:.2f}\n")
        log_file.write("\n" + "="*100 + "\n")
    
    return {
        'ATR_Stop': atr_stop_level,
        'R_Target': target_display,
        'Win_Rate': win_rate,
        'Avg_Winner': avg_winner,
        'Avg_Loser': avg_loser,
        'Expectancy': expectancy,
        'Profit_Factor': profit_factor,
        'Total_Profit': total_profit,
        'Winners': len(winners),
        'Losers': len(losers),
        'Trade_Logs': trade_logs
    }

# Complete optimization ranges - Simplified approach for clean increments
atr_levels = np.arange(0.6, 3.1, 0.1).round(1).tolist()  # 0.6 to 3.0 in 0.1 increments

# Define clean target ranges for both redline and fixed strategies
clean_targets = np.arange(1.0, 5.1, 0.2).round(1).tolist()  # 1.0, 1.2, 1.4, ... 5.0

# Generate all combinations
all_combinations = []

for atr_level in atr_levels:
    # Add redline strategies with clean targets (in current ATR terms)
    for target_r in clean_targets:
        redline_strategy = f"redline_clean_{target_r:.1f}"
        all_combinations.append((atr_level, redline_strategy))
    
    # Add fixed R targets
    for fixed_r in clean_targets:
        all_combinations.append((atr_level, fixed_r))

total_combinations = len(all_combinations)

print(f"🚀 RUNNING COMPLETE OPTIMIZATION WITH FULL LOGGING")
print(f"📊 Testing {total_combinations} combinations")
print(f"🎯 Clean target increments: 1.0R, 1.2R, 1.4R, etc. (no decimals like 4.67R)")
print(f"   • Redline strategies: win_win=actual exits, win_lose=test new targets") 
print(f"   • Fixed R targets: Binary outcome (hit target or -1R)")
print(f"📁 This will generate detailed logs for ALL combinations...")

# Create timestamped filenames
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
detailed_log_filename = f"complete_atr_optimization_detailed_{timestamp}.txt"
summary_filename = f"complete_atr_optimization_summary_{timestamp}.txt"
results_csv_filename = f"complete_atr_optimization_results_{timestamp}.csv"

print(f"📝 Detailed log: {detailed_log_filename}")
print(f"📊 Summary: {summary_filename}")
print(f"📈 Results CSV: {results_csv_filename}")

# Run complete optimization with logging
optimization_results = []

# Open detailed log file
with open(detailed_log_filename, 'w', encoding='utf-8') as detailed_log:
    detailed_log.write("🔍 COMPLETE ATR + R TARGET OPTIMIZATION - DETAILED LOG\n")
    detailed_log.write("=" * 80 + "\n")
    detailed_log.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    detailed_log.write(f"Dataset: {len(valid_enhanced)} valid trades\n")
    detailed_log.write(f"ATR Range: {atr_levels[0]:.1f} to {atr_levels[-1]:.1f} (increments: 0.1)\n")
    detailed_log.write(f"Target Range: {clean_targets[0]:.1f} to {clean_targets[-1]:.1f} (increments: 0.2)\n")
    detailed_log.write(f"Total Combinations: {total_combinations}\n")
    detailed_log.write("=" * 80 + "\n")
    detailed_log.write("🎯 CORRECT REDLINE LOGIC:\n")
    detailed_log.write("   • win_win trades = actual exits at original redline (get full profit)\n") 
    detailed_log.write("   • win_lose trades = missed original redline (test against new target)\n")
    detailed_log.write("   • Targets shown in current ATR terms (clean increments)\n")
    detailed_log.write("🎯 FIXED R TARGETS: Binary outcome (hit exact target = +R, miss target = -1R)\n")
    detailed_log.write("=" * 80 + "\n\n")
    
    # Run all combinations
    for combination_count, (atr_level, r_target) in enumerate(all_combinations, 1):
        
        # Show progress
        if combination_count % 100 == 0:
            print(f"📈 Progress: {combination_count}/{total_combinations} combinations completed ({combination_count/total_combinations*100:.1f}%)")
        
        # Run simulation with logging
        result = simulate_atr_and_r_target_with_logging(
            valid_enhanced, atr_level, r_target, detailed_log
        )
        
        optimization_results.append(result)

print(f"✅ Completed all {total_combinations} combinations!")

# Convert to DataFrame for analysis
opt_df = pd.DataFrame([{k: v for k, v in result.items() if k != 'Trade_Logs'} 
                       for result in optimization_results])

# Save results to CSV with ranking system
ranked_df = opt_df.copy()

# Add ranking columns
ranked_df['Rank_Expectancy'] = ranked_df['Expectancy'].rank(method='min', ascending=False).astype(int)
ranked_df['Rank_Total_Profit'] = ranked_df['Total_Profit'].rank(method='min', ascending=False).astype(int)
ranked_df['Rank_Win_Rate'] = ranked_df['Win_Rate'].rank(method='min', ascending=False).astype(int)
ranked_df['Rank_Profit_Factor'] = ranked_df['Profit_Factor'].rank(method='min', ascending=False).astype(int)

# Sort by expectancy rank (best first)
ranked_df = ranked_df.sort_values('Rank_Expectancy').reset_index(drop=True)

# Reorder columns for better readability
column_order = [
    'Rank_Expectancy', 'ATR_Stop', 'R_Target', 'Expectancy', 'Win_Rate', 'Total_Profit', 
    'Profit_Factor', 'Avg_Winner', 'Avg_Loser', 'Winners', 'Losers',
    'Rank_Total_Profit', 'Rank_Win_Rate', 'Rank_Profit_Factor'
]

ranked_df = ranked_df[column_order]

# Save ranked results to CSV
ranked_df.to_csv(results_csv_filename, index=False)

print(f"💾 CSV saved with ranking system: {results_csv_filename}")

# DEFINE ALL VARIABLES BEFORE USING THEM
# Sort all results by expectancy (highest to lowest)
all_by_expectancy = opt_df.sort_values('Expectancy', ascending=False).reset_index(drop=True)

# Find best results for each metric
best_expectancy = opt_df.loc[opt_df['Expectancy'].idxmax()]
best_profit = opt_df.loc[opt_df['Total_Profit'].idxmax()]
best_winrate = opt_df.loc[opt_df['Win_Rate'].idxmax()]

# Top results for reference sections
top_20_expectancy = opt_df.nlargest(20, 'Expectancy')
top_20_profit = opt_df.nlargest(20, 'Total_Profit')
top_20_winrate = opt_df.nlargest(20, 'Win_Rate')

# Create summary file
with open(summary_filename, 'w', encoding='utf-8') as summary:
    summary.write("🎯 COMPLETE ATR + R TARGET OPTIMIZATION - SUMMARY\n")
    summary.write("=" * 60 + "\n")
    summary.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    summary.write(f"Total combinations tested: {total_combinations}\n")
    summary.write(f"Dataset: {len(valid_enhanced)} valid trades\n")
    summary.write(f"Clean target increments: 1.0R, 1.2R, 1.4R, etc.\n\n")
    
    summary.write("🎯 TARGET EXPLANATIONS:\n")
    summary.write("   • Redline(X.XR): win_win=actual exits, win_lose=test new target\n")
    summary.write("   • Fixed R: Binary outcome (hit exact target or -1R loss)\n\n")
    
    summary.write("🏆 OPTIMAL RESULTS:\n")
    summary.write("-" * 40 + "\n")
    summary.write(f"🎯 Best Expectancy: {best_expectancy['ATR_Stop']:.1f}x ATR + {best_expectancy['R_Target']} target\n")
    summary.write(f"   Expectancy: {best_expectancy['Expectancy']:+.3f}R per trade\n")
    summary.write(f"   Win Rate: {best_expectancy['Win_Rate']:.1f}%\n")
    summary.write(f"   Total Profit: {best_expectancy['Total_Profit']:.2f}R\n\n")
    
    summary.write(f"💰 Highest Total Profit: {best_profit['ATR_Stop']:.1f}x ATR + {best_profit['R_Target']} target\n")
    summary.write(f"   Total Profit: {best_profit['Total_Profit']:.2f}R\n")
    summary.write(f"   Expectancy: {best_profit['Expectancy']:+.3f}R\n")
    summary.write(f"   Win Rate: {best_profit['Win_Rate']:.1f}%\n\n")
    
    summary.write(f"🎪 Highest Win Rate: {best_winrate['ATR_Stop']:.1f}x ATR + {best_winrate['R_Target']} target\n")
    summary.write(f"   Win Rate: {best_winrate['Win_Rate']:.1f}%\n")
    summary.write(f"   Expectancy: {best_winrate['Expectancy']:+.3f}R\n")
    summary.write(f"   Total Profit: {best_winrate['Total_Profit']:.2f}R\n\n")
    
    # All combinations by expectancy
    summary.write("📊 ALL COMBINATIONS BY EXPECTANCY (COMPLETE LIST):\n")
    summary.write("-" * 90 + "\n")
    summary.write(f"{'Rank':<4} {'ATR':<6} {'Target':<15} {'Expectancy':<10} {'Win Rate':<8} {'Total Profit':<12} {'Winners':<7} {'Losers':<6}\n")
    summary.write("-" * 90 + "\n")
    
    for i, (_, row) in enumerate(all_by_expectancy.iterrows(), 1):
        target_str = str(row['R_Target'])[:14]  # Truncate if too long
        summary.write(f"{i:<4} {row['ATR_Stop']:<6.1f} {target_str:<15} {row['Expectancy']:<10.3f} {row['Win_Rate']:<8.1f}% {row['Total_Profit']:<12.2f} {row['Winners']:<7} {row['Losers']:<6}\n")

    # Add summary statistics
    summary.write(f"\n📈 SUMMARY STATISTICS:\n")
    summary.write("-" * 40 + "\n")
    summary.write(f"Best Expectancy: {opt_df['Expectancy'].max():.3f}\n")
    summary.write(f"Worst Expectancy: {opt_df['Expectancy'].min():.3f}\n")
    summary.write(f"Average Expectancy: {opt_df['Expectancy'].mean():.3f}\n")
    summary.write(f"Median Expectancy: {opt_df['Expectancy'].median():.3f}\n")
    summary.write(f"Std Dev Expectancy: {opt_df['Expectancy'].std():.3f}\n")
    
    summary.write(f"\nProfitable Combinations: {len(opt_df[opt_df['Expectancy'] > 0])}/{len(opt_df)} ({len(opt_df[opt_df['Expectancy'] > 0])/len(opt_df)*100:.1f}%)\n")
    summary.write(f"Expectancy > 0.20: {len(opt_df[opt_df['Expectancy'] > 0.20])}/{len(opt_df)} ({len(opt_df[opt_df['Expectancy'] > 0.20])/len(opt_df)*100:.1f}%)\n")
    summary.write(f"Expectancy > 0.15: {len(opt_df[opt_df['Expectancy'] > 0.15])}/{len(opt_df)} ({len(opt_df[opt_df['Expectancy'] > 0.15])/len(opt_df)*100:.1f}%)\n")
    
    # Top 10 by other metrics for reference
    summary.write(f"\n💰 TOP 10 BY TOTAL PROFIT:\n")
    summary.write("-" * 70 + "\n")
    for i, (_, row) in enumerate(top_20_profit.head(10).iterrows(), 1):
        target_str = str(row['R_Target'])[:14]
        summary.write(f"{i:<2} {row['ATR_Stop']:.1f}x {target_str:<15} | Profit: {row['Total_Profit']:.2f}R | Exp: {row['Expectancy']:+.3f}\n")
    
    summary.write(f"\n🎯 TOP 10 BY WIN RATE:\n")
    summary.write("-" * 70 + "\n")
    for i, (_, row) in enumerate(top_20_winrate.head(10).iterrows(), 1):
        target_str = str(row['R_Target'])[:14]
        summary.write(f"{i:<2} {row['ATR_Stop']:.1f}x {target_str:<15} | WR: {row['Win_Rate']:.1f}% | Exp: {row['Expectancy']:+.3f}\n")

print(f"\n🎊 COMPLETE OPTIMIZATION FINISHED!")
print("=" * 50)
print(f"📁 Files created:")
print(f"   • {detailed_log_filename} - Complete trade-by-trade logs")
print(f"   • {summary_filename} - ALL {total_combinations} combinations ranked by expectancy")
print(f"   • {results_csv_filename} - All results in spreadsheet format")

print(f"\n🏆 BEST RESULTS:")
print(f"🎯 Optimal: {best_expectancy['ATR_Stop']:.1f}x ATR + {best_expectancy['R_Target']} target (Expectancy: {best_expectancy['Expectancy']:+.3f})")
print(f"💰 Max Profit: {best_profit['ATR_Stop']:.1f}x ATR + {best_profit['R_Target']} target (Profit: {best_profit['Total_Profit']:.2f}R)")

print(f"\n📊 PERFORMANCE DISTRIBUTION:")
profitable_count = len(opt_df[opt_df['Expectancy'] > 0])
excellent_count = len(opt_df[opt_df['Expectancy'] > 0.20])
print(f"   • Profitable combinations: {profitable_count}/{total_combinations} ({profitable_count/total_combinations*100:.1f}%)")
print(f"   • Excellent (>0.20 expectancy): {excellent_count}/{total_combinations} ({excellent_count/total_combinations*100:.1f}%)")

print(f"\n✨ Check the summary file for complete ranking of all {total_combinations} combinations!")

🚀 RUNNING COMPLETE OPTIMIZATION WITH FULL LOGGING
📊 Testing 1050 combinations
🎯 Clean target increments: 1.0R, 1.2R, 1.4R, etc. (no decimals like 4.67R)
   • Redline strategies: win_win=actual exits, win_lose=test new targets
   • Fixed R targets: Binary outcome (hit target or -1R)
📁 This will generate detailed logs for ALL combinations...
📝 Detailed log: complete_atr_optimization_detailed_20250608_113213.txt
📊 Summary: complete_atr_optimization_summary_20250608_113213.txt
📈 Results CSV: complete_atr_optimization_results_20250608_113213.csv
📈 Progress: 100/1050 combinations completed (9.5%)
📈 Progress: 200/1050 combinations completed (19.0%)
📈 Progress: 300/1050 combinations completed (28.6%)
📈 Progress: 400/1050 combinations completed (38.1%)
📈 Progress: 500/1050 combinations completed (47.6%)
📈 Progress: 600/1050 combinations completed (57.1%)
📈 Progress: 700/1050 combinations completed (66.7%)
📈 Progress: 800/1050 combinations completed (76.2%)
📈 Progress: 900/1050 combinations comp